In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [30]:
design.get_chip_z('Q_chip')

'5um'

In [30]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='20um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8 
    resonator_length_2.append(a)
print(resonator_length_2)


[7.653254683870284, 7.424788339841421, 7.2083465402091305]


# Flipchip

In [31]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-130}um",
                                              pos_y=f"{q0y+100}um",
                                              pad_width="100um",
                                              pad_height="200um",
                                              cpw_stub_height="200um",
                                              orientation='270',                                            
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [32]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [33]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [34]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [35]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [27]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [28]:
hfss.start()

INFO 09:38PM [connect_project]: Connecting to Ansys Desktop API...
INFO 09:38PM [load_ansys_project]: 	Opened Ansys App
INFO 09:38PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 09:38PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project23
INFO 09:38PM [connect_design]: No active design found (or error getting active design).
INFO 09:38PM [connect]: 	 Connected to project "Project23". No design detected


True

In [36]:
hfss.activate_ansys_design("flip_V4", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

09:41PM 20s WARNING [activate_ansys_design]: The design_name=flip_V4 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 09:41PM [connect_design]: 	Opened active design
	Design:    flip_V4 [Solution type: Eigenmode]
WARNING 09:41PM [connect_setup]: 	No design setup detected.
WARNING 09:41PM [connect_setup]: 	Creating eigenmode default setup.
INFO 09:41PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [28]:
hfss.close()

True

# LOM Analysis

In [37]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [38]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.74',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.74',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [40]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 10:56PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpi574kpj4.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5740000000, Maxwell, 1, False
INFO 10:56PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp352zuxlt.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5740000000, Maxwell, 1, False
INFO 10:56PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp3gsmgy7z.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5740000000, Maxwell, 2, False
INFO 10:56PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpf9nv7dxb.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5740000000, Maxwell, 3, False
INFO 10:56PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpcwnh87lm.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5740000000, Maxwell, 4, False
INFO 10:56

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,308.280,-74.60300,-66.710,-106.38000
pad_bot_Q0,-74.603,107.55000,-31.574,-0.21393
pad_top_Q0,-66.710,-31.57400,107.130,-7.76200
quad_coupler_c0,-106.380,-0.21393,-7.762,114.89000


In [41]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.95
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.410854 [GHz]
EC 273.220196 [MHz]
EJ 14.854145 [GHz]
alpha -311.584471 [MHz]
dispersion 5.113350 [KHz]
Lq 10.995564 [nH]
Cq 70.896033 [fF]
T1 241.748625 [us]

**Coupling Properties**

tCqbus1 -3.783249 [fF]
gbus1_in_MHz -54.108842 [MHz]
χ_bus1 -0.652750 [MHz]
1/T1bus1 658.348906 [Hz]
T1bus1 241.748625 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.550924,288.446492,14.854145,-330.447492,8.769787,[-91.93883120859066],[-2.345392714900228],2.345393,91.938831
2,5.940177,333.241528,14.854145,-386.809618,34.28653,[-122.91401047999568],[-8.359624677302707],8.359625,122.914010
3,5.597517,293.615125,14.854145,-336.884482,10.429908,[-84.29143463937032],[-2.1264284583015245],2.126428,84.291435
4,5.59453,293.282243,14.854145,-336.469394,10.31555,[-68.27761713881148],[-1.388395694374392],1.388396,68.277617
5,5.536184,286.822207,14.854145,-328.428164,8.296639,[-61.38828197107495],[-1.021245244743096],1.021245,61.388282
6,5.482046,280.900832,14.854145,-321.081,6.749993,[-55.65009143081246],[-0.7704688916288827],0.770469,55.650091
7,5.455893,278.065302,14.854145,-317.570677,6.100594,[-54.69336907388012],[-0.7145720867555394],0.714572,54.693369
8,5.427878,275.045976,14.854145,-313.838486,5.468109,[-54.87543516885564],[-0.6890201831429164],0.689020,54.875435
9,5.419562,274.153293,14.854145,-312.736158,5.292132,[-54.44216123251773],[-0.6696296210651262],0.669630,54.442161
10,5.410854,273.220196,14.854145,-311.584471,5.11335,[-54.10884212120669],[-0.6527499099060653],0.652750,54.108842


In [17]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
